In [10]:
from openai import OpenAI
from pydantic import BaseModel
from typing import Optional
import json
import os

from dotenv import load_dotenv
load_dotenv(override=True)

client = OpenAI()

In [11]:
client.base_url

URL('https://api.openai.com/v1/')

In [12]:
# Customer Service Routine

system_message = (
    "You are a customer support agent for ACME Inc."
    "Always answer in a sentence or less."
    "Follow the following routine with the user:"
    "1. First, ask probing questions and understand the user's problem deeper.\n"
    " - unless the user has already provided a reason.\n"
    "2. Propose a fix (make one up).\n"
    "3. ONLY if not satesfied, offer a refund.\n"
    "4. If accepted, search for the ID and then execute refund."
    ""
)




In [13]:
def run_full_turn(system_message, messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_message}] + messages,
    )
    message = response.choices[0].message
    messages.append(message)

    if message.content: print("Assistant:", message.content)

    return message




In [ ]:
messages = []
while True:
    user = input("User: ")
    messages.append({"role": "user", "content": user})

    run_full_turn(system_message, messages)

> Output: 

User: Black shoe  
Assistant: Can you please provide more details about the issue you're experiencing with the black shoe?  
User: return  
Assistant: Could you let me know why you'd like to return the shoe?  
User: size  
Assistant: I understand the size is an issue; would a size exchange work for you instead of a return?  
User: only return  
Assistant: If you're sure about the return, I can process that for you. Could you please provide me with your order ID?  
User: 123456  
Assistant: Thank you! I'm processing your return for order ID 12345 now. Please hold on a moment.  
User: Sure  
Assistant: Your refund for order ID 12345 has been successfully processed. You should see the funds reflected in your account shortly. Thank you!  
...  

# Adding the functions as tools

In [14]:
import inspect

def function_to_schema(func) -> dict:
    '''Convert a function to a JSON schema.
    Containing the function name, description, and parameters.
    '''
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }

def sample_function(param_1, param_2, the_third_one: int, some_optional="John Doe"):
    """
    This is my docstring. Call this function when you want.
    """
    print("Hello, world")

schema =  function_to_schema(sample_function)
print(json.dumps(schema, indent=2))

{
  "type": "function",
  "function": {
    "name": "sample_function",
    "description": "This is my docstring. Call this function when you want.",
    "parameters": {
      "type": "object",
      "properties": {
        "param_1": {
          "type": "string"
        },
        "param_2": {
          "type": "string"
        },
        "the_third_one": {
          "type": "integer"
        },
        "some_optional": {
          "type": "string"
        }
      },
      "required": [
        "param_1",
        "param_2",
        "the_third_one"
      ]
    }
  }
}


### defining the tools

In [15]:
def look_up_item(search_query):
    """Use to find item ID.
    Search query can be a description or keywords."""

    # return hard-coded item ID - in reality would be a lookup
    items = {
        'black shoes': 'item_123456789',
        'red dress': 'item_132612938',
        'blue shirt': 'item_987654321',
    }
    item_id = items.get(search_query, 'item_132612938')
    print("Found item:", item_id)

    return item_id

def order_status(product_name):
    "Use to fetch the order status of a product."

    return "shipped"

def place_order(item_name, quantity=1):
    "Use to place an order for a product."

    return "success"


def execute_refund(item_id, reason="not provided"):
    print("\n\n=== Refund Summary ===")
    print(f"Item ID: {item_id}")
    print(f"Reason: {reason}")
    print("=================\n")
    print("Refund execution successful!")
    return "success"

def escalate_to_human(summary):
    """Only call this if explicitly asked to."""
    print("Escalating to human agent...")
    print("\n=== Escalation Report ===")
    print(f"Summary: {summary}")
    print("=========================\n")
    exit()


def execute_order(product, price: int):
    """Price should be in USD."""
    print("\n\n=== Order Summary ===")
    print(f"Product: {product}")
    print(f"Price: ${price}")
    print("=================\n")
    confirm = input("Confirm order? y/n: ").strip().lower()
    if confirm == "y":
        print("Order execution successful!")
        return "Success"
    else:
        print("Order cancelled!")
        return "User cancelled order."


In [6]:
messages = []

tools = [execute_refund, look_up_item, order_status, place_order]
tool_schemas = [function_to_schema(tool) for tool in tools]

In [7]:
response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": "Look up the black boot."}],
            tools=tool_schemas,
        )

response

ChatCompletion(id='chatcmpl-AgSSmSop0kN4UlLxukHVMQ0cKpueK', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zzUQBnHM05mP4F9RNKrKP0SS', function=Function(arguments='{"search_query":"black boot"}', name='look_up_item'), type='function')]))], created=1734682552, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_d02d531b47', usage=CompletionUsage(completion_tokens=18, prompt_tokens=105, total_tokens=123, completion_tokens_details=CompletionTokensDetails(audio_tokens=0, reasoning_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [8]:
len(response.choices), response.choices

(1,
 [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zzUQBnHM05mP4F9RNKrKP0SS', function=Function(arguments='{"search_query":"black boot"}', name='look_up_item'), type='function')]))])

In [9]:
message = response.choices[0].message
message

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zzUQBnHM05mP4F9RNKrKP0SS', function=Function(arguments='{"search_query":"black boot"}', name='look_up_item'), type='function')])

In [10]:
message.tool_calls[0].function

Function(arguments='{"search_query":"black boot"}', name='look_up_item')

In [11]:
response1 = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": "black boot staus."}],
            tools=tool_schemas,
        )

message1 = response1.choices[0].message
message1.tool_calls[0].function


Function(arguments='{"product_name":"black boot"}', name='order_status')

#### Map the function name (provided by llm) with function to execute

In [12]:
tools_map= {tool.__name__: tool for tool in tools}
tools_map

{'execute_refund': <function __main__.execute_refund(item_id, reason='not provided')>,
 'look_up_item': <function __main__.look_up_item(search_query)>,
 'order_status': <function __main__.order_status(product_name)>}

#### Function to Execute the tool

In [14]:
def execute_tool_call(tool_call, tools_map):
    # tool_call = ChatCompletionMessageToolCall(
    #                      id='call_DQ1zjGGOazdpUmt5Px0FJG3W', 
    #                      function=Function(
    #                                   arguments='{"search_query":"black boot"}', 
    #                                   name='look_up_item'), 
    #                      type='function')
    name = tool_call.function.name
    function_to_call = tools_map.get(name)
    
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")


    # call corresponding function with provided arguments
    result = function_to_call(**args)
    return result

execute_tool_call(message.tool_calls[0], tools_map)

Assistant: look_up_item({'search_query': 'black boot'})


'item_132612938'

In [15]:
message.tool_calls

[ChatCompletionMessageToolCall(id='call_zzUQBnHM05mP4F9RNKrKP0SS', function=Function(arguments='{"search_query":"black boot"}', name='look_up_item'), type='function')]

In [16]:
tool_calls = [message.tool_calls[0], message1.tool_calls[0]] 
tool_calls

[ChatCompletionMessageToolCall(id='call_zzUQBnHM05mP4F9RNKrKP0SS', function=Function(arguments='{"search_query":"black boot"}', name='look_up_item'), type='function'),
 ChatCompletionMessageToolCall(id='call_Q1dBPtPvCrv34MW9eNlcSoUc', function=Function(arguments='{"product_name":"black boot"}', name='order_status'), type='function')]

In [17]:
# for tool_call in message.tool_calls:
for tool_call in tool_calls:
    result = execute_tool_call(tool_call, tools_map)

    # add result back to conversation 
    result_message = {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": result,
    }
    print(result_message)
    messages.append(result_message)

Assistant: look_up_item({'search_query': 'black boot'})
{'role': 'tool', 'tool_call_id': 'call_zzUQBnHM05mP4F9RNKrKP0SS', 'content': 'item_132612938'}
Assistant: order_status({'product_name': 'black boot'})
{'role': 'tool', 'tool_call_id': 'call_Q1dBPtPvCrv34MW9eNlcSoUc', 'content': 'shipped'}


### Calling tools back to back in chain

In [18]:
tools

[<function __main__.execute_refund(item_id, reason='not provided')>,
 <function __main__.look_up_item(search_query)>,
 <function __main__.order_status(product_name)>]

In [19]:
tool_schemas

[{'type': 'function',
  'function': {'name': 'execute_refund',
   'description': '',
   'parameters': {'type': 'object',
    'properties': {'item_id': {'type': 'string'},
     'reason': {'type': 'string'}},
    'required': ['item_id']}}},
 {'type': 'function',
  'function': {'name': 'look_up_item',
   'description': 'Use to find item ID.\n    Search query can be a description or keywords.',
   'parameters': {'type': 'object',
    'properties': {'search_query': {'type': 'string'}},
    'required': ['search_query']}}},
 {'type': 'function',
  'function': {'name': 'order_status',
   'description': 'Use to fetch the order status of a product.',
   'parameters': {'type': 'object',
    'properties': {'product_name': {'type': 'string'}},
    'required': ['product_name']}}}]

In [20]:
tools_map

{'execute_refund': <function __main__.execute_refund(item_id, reason='not provided')>,
 'look_up_item': <function __main__.look_up_item(search_query)>,
 'order_status': <function __main__.order_status(product_name)>}

In [50]:
messages

[{'role': 'tool',
  'tool_call_id': 'call_DQ1zjGGOazdpUmt5Px0FJG3W',
  'content': 'item_132612938'},
 {'role': 'tool',
  'tool_call_id': 'call_fVHxYY36qMFc5Vrsd61UT3sH',
  'content': 'shipped'}]

In [21]:
def run_full_turn(system_message, tools, messages):

    num_init_messages = len(messages)

    # messages will contain all the conversation history
    # loops until all tool calls are handled back to back by the assistant
    # loop breaks when assistant has no more tool calls to make, though it can still make regular responses and saved to messages
    while True:

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": system_message}] + messages,
            tools=tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)
        print("-------------------------")
        print(message)
        print("-------------------------")
        if message.content:  # print assistant response
            print("Assistant:", message.content)

        #--------------------------------------------------------------------------- loop exit condition
        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools_map)

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            print("Tool response:", result_message)
            messages.append(result_message)
        #---------------------------------------------------------------------------

    # ==== 3. return new messages =====
    return messages[num_init_messages:]


def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Execute tool: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)




In [ ]:
messages = []
while True:
    user = input("User: ")
    messages.append({"role": "user", "content": user})

    new_messages = run_full_turn(system_message, tools, messages)
    messages.extend(new_messages)

> Output:

`User: Black shoe`  
Assistant: Could you please provide more details about the issue you're experiencing with the black shoe?  
`User: details`  
Assistant: Are you having problems with the fit, quality, or a specific feature of the black shoe?  
`User: delivery status`  
Execute tool: order_status({'product_name': 'black shoe'})  
Assistant: Your black shoe has been shipped. Would you like assistance with any specific issues related to it?  
`User: item detail`  
Execute tool: look_up_item({'search_query': 'black shoe'})  
Assistant: The item ID for the black shoe is 132612938. Please let me know if you have any specific questions about it!  
`User: return`    
Assistant: Could you please specify the reason for the return?  
`User: size`
Execute tool: execute_refund({'item_id': '132612938', 'reason': 'return'})  
Refund Summary: 132612938 return  
Assistant: Your return has been processed successfully, and a refund has been issued. If you need further assistance, feel free to ask!  
`User: Thank you!`
Assistant: You're welcome! Have a great day!  

# HandOff

In [16]:
class Agent(BaseModel):
    name: str = "Agent"
    model: str = "gpt-4o-mini"
    instructions: str = "You are a helpful Agent"
    tools: list = []

In [17]:
def run_full_turn(agent, messages):

    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        agentic_tool_schemas = [function_to_schema(tool) for tool in agent.tools]
        agentic_tools_map = {tool.__name__: tool for tool in agent.tools}

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model=agent.model,
            messages=[{"role": "system", "content": agent.instructions}] + messages,
            tools=agentic_tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)
        print("-------------------------")
        print(message)
        print("-------------------------")
        if message.content:  # print assistant response
            print("Assistant:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, agentic_tools_map)

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            print("Tool response:", result_message)
            messages.append(result_message)

    # ==== 3. return new messages =====
    return messages[num_init_messages:]


def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"execute tool: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)

#### Define the agents and their roles
1. Agent 1: Refund Agent
2. Agent 2: Place Order Agent

In [18]:
# ******** the function already defined above *********
# def execute_refund(item_name):
#     return "success"
# -----------------------------------------------------


# refund agent will have only one tool - execute_refund
refund_agent = Agent(
    name="Refund Agent",
    instructions="You are a refund agent. Help the user with refunds.",
    tools=[execute_refund],
)

# ******** the function already defined above *********
# def place_order(item_name):
    # return "success"
# -----------------------------------------------------

sales_assistant = Agent(
    name="Sales Assistant",
    instructions="You are a sales assistant. Sell the user a product.",
    tools=[place_order],
)


In [20]:
messages = []
user_query = "Place an order for a black boot."
print("User:", user_query)
messages.append({"role": "user", "content": user_query})

response = run_full_turn(sales_assistant, messages) # sales assistant
messages.extend(response)


user_query = "Actually, I want a refund." # implitly refers to the last item
print("User:", user_query)
messages.append({"role": "user", "content": user_query})
response = run_full_turn(refund_agent, messages) # refund agent

User: Place an order for a black boot.
-------------------------
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_racgNCmaKarvviFLGNUSoeQG', function=Function(arguments='{"item_name":"black boot"}', name='place_order'), type='function')])
-------------------------
execute tool: place_order({'item_name': 'black boot'})
Tool response: {'role': 'tool', 'tool_call_id': 'call_racgNCmaKarvviFLGNUSoeQG', 'content': 'success'}
-------------------------
ChatCompletionMessage(content='Your order for a black boot has been successfully placed! If you need anything else, just let me know.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
-------------------------
Assistant: Your order for a black boot has been successfully placed! If you need anything else, just let me know.
User: Actually, I want a refund.
-------------------------
ChatCompletionMessage(content=Non

## LLLM based HandOff (automated)

In [21]:
class Response(BaseModel):
    agent: Optional[Agent]
    messages: list

In [22]:
def run_full_turn(agent, messages):

    current_agent = agent
    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        agentic_tool_schemas = [function_to_schema(tool) for tool in current_agent.tools]
        agentic_tools_map = {tool.__name__: tool for tool in current_agent.tools}

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model=agent.model,
            messages=[{"role": "system", "content": current_agent.instructions}]
            + messages,
            tools=agentic_tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)
        print("-------------------------")
        print(message, flush=True)
        print("-------------------------")
        if message.content:  # print agent response
            print(f"{current_agent.name}:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, agentic_tools_map, current_agent.name)

            if type(result) is Agent:  # if agent transfer, update current agent
                print("************** Tool change requested **************")
                current_agent = result
                result = (
                    f"Transfered to {current_agent.name}. Adopt persona immediately."
                )

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            print("Tool response:", result_message, flush=True)
            messages.append(result_message)

    # ==== 3. return last agent used and new messages =====
    return Response(agent=current_agent, messages=messages[num_init_messages:])


def execute_tool_call(tool_call, tools, agent_name):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Execute tool - {agent_name}:", f"{name}({args})")

    return tools[name](**args)  # call corresponding function with provided arguments

### Define the more agents and handoffs

#### AGENTS
1. Agent 1: Refund Agent
2. Agent 2: Sales Assistant
3. Agent 3: Triage Agent 
4. Agent 4: Sales Agent
5. Agent 5: Issues and Repairs Agent
  

#### HANDOFFS

1. HandOff 1: Refund Agent 
2. HandOff 2: Sales Agent
3. HandOff 3: Triage Agent
4. HandOff 4: Issues and Repairs Agent

In [23]:
def transfer_to_refunds() -> Agent:
    return refund_agent

def transfer_to_sales_agent():
    """User for anything sales or buying related."""
    return sales_agent


def transfer_to_issues_and_repairs():
    """User for issues, repairs, or refunds."""
    return issues_and_repairs_agent


def transfer_back_to_triage():
    """Call this if the user brings up a topic outside of your purview,
    including escalating to human."""
    return triage_agent



# refund agent will have only one tool - execute_refund
refund_agent = Agent(
    name="Refund Agent",
    instructions="You are a refund agent. Help the user with refunds.",
    tools=[execute_refund],
)

sales_assistant = Agent(
    name="Sales Assistant",
    instructions="You are a sales assistant. Sell the user a product.",
    tools=[place_order],
)

triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "You are a customer service bot for ACME Inc. "
        "Introduce yourself. Always be very brief. "
        "Gather information to direct the customer to the right department. "
        "But make your questions subtle and natural."
    ),
    tools=[transfer_to_sales_agent, transfer_to_issues_and_repairs, escalate_to_human, transfer_to_refunds],
)


sales_agent = Agent(
    name="Sales Agent",
    instructions=(
        "You are a sales agent for ACME Inc."
        "Always answer in a sentence or less."
        "Follow the following routine with the user:"
        "1. Ask them about any problems in their life related to catching roadrunners.\n"
        "2. Casually mention one of ACME's crazy made-up products can help.\n"
        " - Don't mention price.\n"
        "3. Once the user is bought in, drop a ridiculous price.\n"
        "4. Only after everything, and if the user says yes, "
        "tell them a crazy caveat and execute their order.\n"
        ""
    ),
    tools=[execute_order, transfer_back_to_triage],
)

issues_and_repairs_agent = Agent(
    name="Issues and Repairs Agent",
    instructions=(
        "You are a customer support agent for ACME Inc."
        "Always answer in a sentence or less."
        "Follow the following routine with the user:"
        "1. First, ask probing questions and understand the user's problem deeper.\n"
        " - unless the user has already provided a reason.\n"
        "2. Propose a fix (make one up).\n"
        "3. ONLY if not satesfied, offer a refund.\n"
        "4. If accepted, search for the ID and then execute refund."
        ""
    ),
    tools=[transfer_to_refunds, look_up_item, transfer_back_to_triage],
)

In [24]:
agent = triage_agent
messages = []

while True:
    user = input("User: ")
    print(f"`User: {user}` \n", flush=True)
    messages.append({"role": "user", "content": user})

    response = run_full_turn(agent, messages)
    agent = response.agent
    messages.extend(response.messages)

`User: black shoes` 

-------------------------
ChatCompletionMessage(content='Are you looking to purchase black shoes, or do you need help with an issue related to them?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
-------------------------
Triage Agent: Are you looking to purchase black shoes, or do you need help with an issue related to them?
`User: new purchase` 

-------------------------
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_My881WYTXipihapc6N6tpw6T', function=Function(arguments='{}', name='transfer_to_sales_agent'), type='function')])
-------------------------
Execute tool - Triage Agent: transfer_to_sales_agent({})
************** Tool change requested **************
Tool response: {'role': 'tool', 'tool_call_id': 'call_My881WYTXipihapc6N6tpw6T', 'content': 'Transfered to Sales Agent. Adopt persona immediately.'}
-----------------

KeyboardInterrupt: Interrupted by user

In [ ]:
query + prompt (list of data variable that can be fetched from apis)
-> list down all the variable that are required to resolve the query


llm -> list of variable or None


map = {variable <-> apis}
run apis -> "additional info"




In [ ]:

1. API based answer
# 2. Pure Rag based answer (from the doc)   (existing pipeline)
3. HYbrid answer (API + Rag)
    i. API fetch prerequisite data  + query
    ii. Rag generate response (docs)


query -> [api based pipeline]  (query + "additional details" or None) -> [article bot pipeline] -> response






n api variables

for a query if we require some variable to be fetched from api
then we must have the related input variables


k categories
based of api inputs and outputs

input variables: 
    - user details
    - for chat 

INPUT_DETAILS_AGENTS


NEXT_API_PROCESS_AGENT


OUTPUT_AGENT (RAG decision maker)

query
SAQ/Intent(query + History)
Details
API response







n APIs
api1 : order id
api2 : product id
api3 : all the dates
...


query -> [api1, api3, ...]

query:
I want a refund?

Additional details:
1. order id
2. Product id
3. Product order date


RAG (query): DOCS

----------
Response:
Modified query = Query + Additional details
Article
Transcript
Policy











---------------------------

{
    order_id: 123456,
    product_name: 'black shoe',
    issue: 'size',
}

Use case:

I have ordered 2 phones in one order 

Refund


AGENT ( order id, product id,     <- naive apis 
        api-x (number of phone)   <- specific to use case
      )

APIS;
- order id
- price
- product id
- dates
- purchase details

N = 7, k = 3, Z = 2

S = min coin for 1 way 
Extra coins = N - S

S = (Z+1)*(k-1) + 1


1 2 3 4 5 6 7
1     4     7
(0 extra ways)

7//3 = 2 = (k-1)
7-1%3 = 1

1 2 3 4 5 6 7 8
1     4     7
1     4       8
1       5     8
  2     5     8
(3 extra ways) = k^(extra coins) = 3^1 = 3

8//3 = 2 = (k-1)
8%3 = 2


1 2 3 4 5 6 7 8 9 
1     4     7
1     4       8
1     4         9
1       5     8
1       5       9
1         6     9 
  2     5     8
  2     5       9
  2      6      9
    3     6     9
(9 extra ways) = k^(extra coins) = 3^2 = 9


9//3 = 3 > (k-1)
9%3 = 0
9//2 = 4 
9%2 = 1


In [ ]:
N = 13, k =4, Z = 2

S = (Z+1) * (k-1) + 1
S = 3 * 3 + 1 = 10

1 2 3 4 5 6 7 8 9 10
1     4     7     10 


In [6]:
def identicalCoins(N, K, Z):
    S = (Z+1) * (K-1) + 1
    if S > N:
        return -404
    if S == N:
        return 1
    else:
        extra = N - S
        print(extra)
        return  (1 + K**extra ) % 1000000007

identicalCoins(7, 3, 2)

1

In [7]:
def f (**kwargs):
    m = kwargs.get("a", 's')
    print(m)


args = {"a": 1, "b": 2, "c": 3}
f(**args)

1


In [29]:
response =""" ```json
          
                        {
                        "What policies does the user have?": ["policy_id", "policy_name"],
                        "What is the premium amount for each policy?": ["policy_id", "premium_amount", "premium_breakdown", "amount_paid"],
                        "What is the due date of the premium for each policy?": ["policy_id", "premium_due_date", "penalty_charges"]
                        }
                        
```
"""
import json

if '```json' in response:
    # remove the code block
    response = re.sub(r'```json\n?', '', response)
if '```' in response:
    response = re.sub(r'```', '', response)
json_data = json.loads(response)



json_data

{'What policies does the user have?': ['policy_id', 'policy_name'],
 'What is the premium amount for each policy?': ['policy_id',
  'premium_amount',
  'premium_breakdown',
  'amount_paid'],
 'What is the due date of the premium for each policy?': ['policy_id',
  'premium_due_date',
  'penalty_charges']}

In [31]:
import os
import shutil

# Define paths
source_path = r"E:\TO BACKUP\WhatsApp Stickers"
destination_path = r"E:\TO BACKUP\dummy"

# Create destination directory if it doesn't exist
os.makedirs(destination_path, exist_ok=True)

# Iterate through all files in the source directory
for file_name in os.listdir(source_path):
    if file_name.endswith(".webp"):  # Check if file has .webp extension
        # Construct full file paths
        source_file = os.path.join(source_path, file_name)
        destination_file = os.path.join(destination_path, file_name.replace(".webp", ".png"))
        
        # Copy and rename the file
        shutil.copy(source_file, destination_file)

print(f"All files copied and renamed to: {destination_path}")


All files copied and renamed to: E:\TO BACKUP\dummy


In [32]:
import os
import shutil
import math

# Get the list of all files in the destination directory
all_files = [f for f in os.listdir(destination_path) if os.path.isfile(os.path.join(destination_path, f))]

# Calculate the number of files per folder
num_files = len(all_files)
files_per_folder = math.ceil(num_files / 10)

# Create subfolders and move files
for i in range(10):
    subfolder_path = os.path.join(destination_path, f"subfolder_{i+1}")
    os.makedirs(subfolder_path, exist_ok=True)
    
    for j in range(files_per_folder):
        if all_files:
            file_to_move = all_files.pop(0)
            shutil.move(os.path.join(destination_path, file_to_move), os.path.join(subfolder_path, file_to_move))

print(f"All files have been moved to 10 subfolders in {destination_path}")

All files have been moved to 10 subfolders in E:\TO BACKUP\dummy


In [33]:
# zip all the folders in destination_path
import zipfile

# Define the name of the zip file
zip_file = os.path.join(destination_path, "subfolders.zip")

# Create a ZipFile object
with zipfile.ZipFile(zip_file, "w") as zipf:
    # Iterate over all subfolders
    for folder_name in os.listdir(destination_path):
        folder_path = os.path.join(destination_path, folder_name)
        # Add the folder and its contents to the zip file
        zipf.write(folder_path, folder_name)


In [36]:
import math
n = 9
chess = [[-1]*n]*n  


def bishop(i,j,k,l):
    if i<0 or i>=n or j<0 or j>=n:
        return 100000
    if chess[i][j] != -1:
        return chess[i][j]
    if i == k and j == l:
        return 0
    
    chess[i][j] = 1+ min(
        bishop(i+2, j+1,k,l),
        bishop(i+2, j-1,k,l),
        bishop(i-2, j+1,k,l),
        bishop(i-2, j-1,k,l),
        bishop(i+1, j+2,k,l),
        bishop(i+1, j-2,k,l),
        bishop(i-1, j+2,k,l),
        bishop(i-1, j-2,k,l),
    )
    return chess[i][j]

bishop(4,4,4,8)


RecursionError: maximum recursion depth exceeded

In [37]:
from collections import deque

def bishop_moves(start, end, n):
    directions = [
        (2, 1), (2, -1), (-2, 1), (-2, -1),
        (1, 2), (1, -2), (-1, 2), (-1, -2)
    ]
    
    queue = deque([(start[0], start[1], 0)])  # (x, y, steps)
    visited = set()
    visited.add((start[0], start[1]))
    
    while queue:
        x, y, steps = queue.popleft()
        
        if (x, y) == end:
            return steps
        
        # Explore all valid moves
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n and (nx, ny) not in visited:
                visited.add((nx, ny))
                queue.append((nx, ny, steps + 1))
    
    # If the end position is not reachable, return -1
    return -1

n = 9  # Chessboard size
start = (4, 4)
end = (4, 8)
result = bishop_moves(start, end, n)
print(f"Minimum steps to reach destination: {result}")


Minimum steps to reach destination: 2


In [46]:
def get_maximum_sum(no_adjacent, one_adjacent, both_adjacent):
    n = len(no_adjacent)
    
    # Initialize the dp table
    dp = [[-1] * 3 for _ in range(n + 1)]
    
    # Base cases
    dp[1][0] = no_adjacent[0]
    dp[1][1] = one_adjacent[0]
    dp[1][2] = both_adjacent[0]
    
    # Fill the dp table
    for i in range(2, n + 1):
        dp[i][0] = max(dp[i-1][0] + no_adjacent[i-1], dp[i-1][1] + no_adjacent[i-1])
        dp[i][1] = max(
            dp[i-1][0] + one_adjacent[i-1], 
            dp[i-1][1] + one_adjacent[i-1], 
            dp[i-1][2] + no_adjacent[i-1]
        )
        dp[i][2] = max(dp[i-1][1] + both_adjacent[i-1], dp[i-1][2] + one_adjacent[i-1])
    
    # Return the maximum value from the last row of dp
    for i in range(len(dp)):
        print(dp[i])

    return max(dp[n][0], dp[n][1], dp[n][2])

n = 4
no_adjacent = [1,2,3,4]
one_adjacent = [4,4,2,1]
both_adjacent = [0,1,1,0]

# Get the result
result = get_maximum_sum(no_adjacent, one_adjacent, both_adjacent)

# Output the result
print(result)


[-1, -1, -1]
[1, 4, 0]
[6, 8, 5]
[11, 10, 9]
[15, 13, 10]
15


In [ ]:
def maxEfficiency(n, no_adjacent, one_adjacent, both_adjacent):
    dp = {}  # State memoization
    
def solve(mask, last):
    if mask == (1 &lt;&lt; n) - 1:  # All processors deployed
        return 0
        
    if (mask, last) in dp:
        return dp[(mask, last)]
        
    max_eff = 0
    for i in range(n):
        if mask &amp; (1 &lt;&lt; i):  # Already deployed
            continue
            
        # Count adjacent deployed processors
        adj_count = 0
        if i &gt; 0 and mask &amp; (1 &lt;&lt; (i-1)): adj_count += 1
        if i &lt; n-1 and mask &amp; (1 &lt;&lt; (i+1)): adj_count += 1
        
        # Calculate efficiency based on adjacency
        eff = 0
        if adj_count == 0: eff = no_adjacent[i]
        elif adj_count == 1: eff = one_adjacent[i]
        else: eff = both_adjacent[i]
        
        new_mask = mask | (1 &lt;&lt; i)
        max_eff = max(max_eff, eff + solve(new_mask, i))
        
    dp[(mask, last)] = max_eff
    return max_eff



In [47]:
def knapsack_variant(no_adjacent, one_adjacent, both_adjacent):
    n = len(no_adjacent)
    
    # Initialize the dp table
    dp = [[0] * 3 for _ in range(n + 1)]  # 3 states: no neighbor, one neighbor, both neighbors selected
    
    # Base cases
    dp[1][0] = no_adjacent[0]  # If no neighbors selected
    dp[1][1] = one_adjacent[0]  # If one neighbor selected
    dp[1][2] = both_adjacent[0]  # If both neighbors selected
    
    # Fill the dp table
    for i in range(2, n + 1):
        # Current slot with no neighbors selected
        dp[i][0] = max(dp[i-1][0] + no_adjacent[i-1], dp[i-1][1] + no_adjacent[i-1])
        
        # Current slot with one neighbor selected
        dp[i][1] = max(
            dp[i-1][0] + one_adjacent[i-1], 
            dp[i-1][1] + one_adjacent[i-1], 
            dp[i-1][2] + no_adjacent[i-1]
        )
        
        # Current slot with both neighbors selected
        dp[i][2] = max(dp[i-1][1] + both_adjacent[i-1], dp[i-1][2] + one_adjacent[i-1])
    
    # Print the dp table for debugging
    for i in range(len(dp)):
        print(f"DP state at slot {i}: {dp[i]}")

    # Return the maximum value from the last slot across all states
    return max(dp[n][0], dp[n][1], dp[n][2])

# Example input
n = 4
no_adjacent = [1, 2, 3, 4]
one_adjacent = [4, 4, 2, 1]
both_adjacent = [0, 1, 1, 0]

# Get the result
result = knapsack_variant(no_adjacent, one_adjacent, both_adjacent)

# Output the result
print(f"Maximum sum: {result}")


DP state at slot 0: [0, 0, 0]
DP state at slot 1: [1, 4, 0]
DP state at slot 2: [6, 8, 5]
DP state at slot 3: [11, 10, 9]
DP state at slot 4: [15, 13, 10]
Maximum sum: 15


In [77]:
from itertools import permutations, combinations

def generate_all_orders(n):
    return list(permutations(range(n), n))

# Example usage
n = 4
result = generate_all_orders(n)

no_adjacent = [1, 2, 3, 4]
one_adjacent = [4, 4, 2, 1]
both_adjacent = [0, 1, 1, 0]

d = {
    0 : no_adjacent,
    1 : one_adjacent,
    2 : both_adjacent
}

def find_adjacency(a,i):
    if i == 0:
        if a[i+1] == -1:
            return 0
        else:
            return 1
    if i == (len(a)-1):
        if a[i-1] == -1:
            return 0
        else:
            return 1
    if a[i-1] == -1 and a[i+1] == -1:
        return 0
    if a[i-1] != -1 and a[i+1] != -1:
        return 2
    return 1



maxx = -1
for row in result:
    array = [-1]*n
    s_max = 0
    print(row)
    for k in row:
        array[k] = 0
        adj = find_adjacency(array,k)
        print(f"ADJ of {row} : {k} = {adj}")
        s_max += d[adj][k]
    print('----------')
    print(row, s_max)
    print('---------')
    maxx = max(maxx, s_max)

maxx


(0, 1, 2, 3)
ADJ of (0, 1, 2, 3) : 0 = 0
ADJ of (0, 1, 2, 3) : 1 = 1
ADJ of (0, 1, 2, 3) : 2 = 1
ADJ of (0, 1, 2, 3) : 3 = 1
----------
(0, 1, 2, 3) 8
---------
(0, 1, 3, 2)
ADJ of (0, 1, 3, 2) : 0 = 0
ADJ of (0, 1, 3, 2) : 1 = 1
ADJ of (0, 1, 3, 2) : 3 = 0
ADJ of (0, 1, 3, 2) : 2 = 2
----------
(0, 1, 3, 2) 10
---------
(0, 2, 1, 3)
ADJ of (0, 2, 1, 3) : 0 = 0
ADJ of (0, 2, 1, 3) : 2 = 0
ADJ of (0, 2, 1, 3) : 1 = 2
ADJ of (0, 2, 1, 3) : 3 = 1
----------
(0, 2, 1, 3) 6
---------
(0, 2, 3, 1)
ADJ of (0, 2, 3, 1) : 0 = 0
ADJ of (0, 2, 3, 1) : 2 = 0
ADJ of (0, 2, 3, 1) : 3 = 1
ADJ of (0, 2, 3, 1) : 1 = 2
----------
(0, 2, 3, 1) 6
---------
(0, 3, 1, 2)
ADJ of (0, 3, 1, 2) : 0 = 0
ADJ of (0, 3, 1, 2) : 3 = 0
ADJ of (0, 3, 1, 2) : 1 = 1
ADJ of (0, 3, 1, 2) : 2 = 2
----------
(0, 3, 1, 2) 10
---------
(0, 3, 2, 1)
ADJ of (0, 3, 2, 1) : 0 = 0
ADJ of (0, 3, 2, 1) : 3 = 0
ADJ of (0, 3, 2, 1) : 2 = 1
ADJ of (0, 3, 2, 1) : 1 = 2
----------
(0, 3, 2, 1) 8
---------
(1, 0, 2, 3)
ADJ of (1, 0, 2, 3)

14

In [8]:
data = """```json\n{\n  \"QUESTION TYPE\": [\"MULTIPLE\", \"MULTIPLE\", \"MULTIPLE\", \"MULTIPLE\", \"MULTIPLE\", \"MULTIPLE\", \"MULTIPLE\", \"MULTIPLE\", \"MULTIPLE\", \"MULTIPLE\"],\n  \"QUESTION\": [\n    \"What was the scoreline of Manchester United's defeat by Bournemouth?\",\n    \"How many goals did Bournemouth score in quick succession?\",\n    \"Who scored Bournemouth's second goal?\",\n    \"What position was Manchester United in on Christmas Day after their defeat?\",\n    \"Who was responsible for coaching Manchester United's set pieces?\",\n    \"What was the main reason for Manchester United's nervousness according to Ruben Amorim?\",\n    \"Who scored the opening goal for Bournemouth?\",\n    \"What was the reaction of Manchester United's players after going 2-0 down?\",\n    \"What percentage of Bournemouth's goals are scored when the ball is in the opponents' half?\",\n    \"What did Ruben Amorim say about the responsibility for improving set pieces?\"\n  ],\n  \"Question Code\": [\"Q1\", \"Q2\", \"Q3\", \"Q4\", \"Q5\", \"Q6\", \"Q7\", \"Q8\", \"Q9\", \"Q10\"],\n  \"Vertical\": [\"English\", \"English\", \"English\", \"English\", \"English\", \"English\", \"English\", \"English\", \"English\", \"English\"],\n  \"OPTION1\": [\"1-0\", \"1 goal\", \"Justin Kluivert\", \"10th\", \"Carlos Fernandes\", \"Lack of training\", \"Dean Huijsen\", \"Try to score immediately\", \"40%\", \"Carlos Fernandes\"],\n  \"OPTION2\": [\"2-0\", \"2 goals\", \"Dean Huijsen\", \"13th\", \"Ruben Amorim\", \"Anxiety from fans\", \"Antoine Semenyo\", \"Keep the ball for a moment\", \"50%\", \"Ruben Amorim\"],\n  \"OPTION3\": [\"3-0\", \"3 goals\", \"Antoine Semenyo\", \"15th\", \"Assistant Coach\", \"Injury issues\", \"Carlos Fernandes\", \"Defend tightly\", \"60%\", \"The players\"],\n  \"OPTION4\": [\"4-0\", \"4 goals\", \"Carlos Fernandes\", \"20th\", \"Team Captain\", \"Weather conditions\", \"Justin Kluivert\", \"Play defensively\", \"70%\", \"The fans\"],\n  \"OPTION5\": [\"5-0\", \"5 goals\", \"Justin Bieber\", \"18th\", \"Fitness Coach\", \"Poor tactics\", \"Ruben Amorim\", \"Attack aggressively\", \"80%\", \"The management\"],\n  \"CORRECTOPTION\": [\"3-0\", \"2 goals\", \"Justin Kluivert\", \"13th\", \"Ruben Amorim\", \"Anxiety from fans\", \"Dean Huijsen\", \"Keep the ball for a moment\", \"60%\", \"Ruben Amorim\"],\n  \"correct_option_type\": [\"ANY\", \"ANY\", \"ANY\", \"ANY\", \"ANY\", \"ANY\", \"ANY\", \"ANY\", \"ANY\", \"ANY\"],\n  \"MARKS\": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],\n  \"Chapter Name (Category)\": [\"\", \"\", \"\", \"\", \"\", \"\", \"\", \"\", \"\", \"\"],\n  \"REASON For the option being the correct answer\": [\"\", \"\", \"\", \"\", \"\", \"\", \"\", \"\", \"\", \"\"],\n  \"Hint\": [\"\", \"\", \"\", \"\", \"\", \"\", \"\", \"\", \"\", \"\"]\n}\n```"""
import csv
import json
import re

def process_jsonj(response):
    # response format: "{'var': 'value', 'var': 'value'}"
    if "```json" in response:
        response = re.sub(r"```json\n?", "", response)
    if "```" in response:
        response = re.sub(r"```", "", response)
    
    print(response)
    json_data = json.loads(response)



    # Transpose the JSON data to rows
    rows = zip(*json_data.values())

    # Create the CSV file
    with open("output.csv", "w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Write the header
        csv_writer.writerow(json_data.keys())
        
        # Write the rows
        csv_writer.writerows(rows)

    print("CSV file 'output.csv' created successfully!")

    return json_data

process_jsonj(data)

{
  "QUESTION TYPE": ["MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE"],
  "QUESTION": [
    "What was the scoreline of Manchester United's defeat by Bournemouth?",
    "How many goals did Bournemouth score in quick succession?",
    "Who scored Bournemouth's second goal?",
    "What position was Manchester United in on Christmas Day after their defeat?",
    "Who was responsible for coaching Manchester United's set pieces?",
    "What was the main reason for Manchester United's nervousness according to Ruben Amorim?",
    "Who scored the opening goal for Bournemouth?",
    "What was the reaction of Manchester United's players after going 2-0 down?",
    "What percentage of Bournemouth's goals are scored when the ball is in the opponents' half?",
    "What did Ruben Amorim say about the responsibility for improving set pieces?"
  ],
  "Question Code": ["Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10"],
  "

{'QUESTION TYPE': ['MULTIPLE',
  'MULTIPLE',
  'MULTIPLE',
  'MULTIPLE',
  'MULTIPLE',
  'MULTIPLE',
  'MULTIPLE',
  'MULTIPLE',
  'MULTIPLE',
  'MULTIPLE'],
 'QUESTION': ["What was the scoreline of Manchester United's defeat by Bournemouth?",
  'How many goals did Bournemouth score in quick succession?',
  "Who scored Bournemouth's second goal?",
  'What position was Manchester United in on Christmas Day after their defeat?',
  "Who was responsible for coaching Manchester United's set pieces?",
  "What was the main reason for Manchester United's nervousness according to Ruben Amorim?",
  'Who scored the opening goal for Bournemouth?',
  "What was the reaction of Manchester United's players after going 2-0 down?",
  "What percentage of Bournemouth's goals are scored when the ball is in the opponents' half?",
  'What did Ruben Amorim say about the responsibility for improving set pieces?'],
 'Question Code': ['Q1',
  'Q2',
  'Q3',
  'Q4',
  'Q5',
  'Q6',
  'Q7',
  'Q8',
  'Q9',
  'Q10']

In [ ]:
const fs = require("fs");
const path = require("path");

function processJsonToCsv(response) {
  // Remove Markdown code block if present
  response = response.replace(/```json\n?/g, "").replace(/```/g, "");

  let jsonData;
  try {
    jsonData = JSON.parse(response);
  } catch (error) {
    console.error("Error parsing JSON:", error.message);
    return;
  }

  // Transpose the JSON data to rows
  const keys = Object.keys(jsonData);
  const rows = [];
  const numRows = jsonData[keys[0]].length;

  for (let i = 0; i < numRows; i++) {
    const row = keys.map((key) => jsonData[key][i] || ""); // Fill missing values with an empty string
    rows.push(row);
  }

  // Create the CSV content
  let csvContent = keys.join(",") + "\n"; // Header row
  rows.forEach((row) => {
    csvContent += row.map((value) => `"${value}"`).join(",") + "\n"; // Quote values to handle commas in data
  });

  // Write the CSV content to a file
  const outputPath = path.join(__dirname, "output.csv");
  fs.writeFile(outputPath, csvContent, (err) => {
    if (err) {
      console.error("Error writing CSV file:", err.message);
      return;
    }
    console.log(`CSV file 'output.csv' created successfully at ${outputPath}`);
  });
}

// Input data
const data = `{
  "QUESTION TYPE": ["MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE", "MULTIPLE"],
  "QUESTION": [
    "What was the scoreline of Manchester United's defeat by Bournemouth?",
    "How many goals did Bournemouth score in quick succession?",
    "Who scored Bournemouth's second goal?",
    "What position was Manchester United in on Christmas Day after their defeat?",
    "Who was responsible for coaching Manchester United's set pieces?",
    "What was the main reason for Manchester United's nervousness according to Ruben Amorim?",
    "Who scored the opening goal for Bournemouth?",
    "What was the reaction of Manchester United's players after going 2-0 down?",
    "What percentage of Bournemouth's goals are scored when the ball is in the opponents' half?",
    "What did Ruben Amorim say about the responsibility for improving set pieces?"
  ],
  "Question Code": ["Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10"],
  "Vertical": ["English", "English", "English", "English", "English", "English", "English", "English", "English", "English"],
  "OPTION1": ["1-0", "1 goal", "Justin Kluivert", "10th", "Carlos Fernandes", "Lack of training", "Dean Huijsen", "Try to score immediately", "40%", "Carlos Fernandes"],
  "OPTION2": ["2-0", "2 goals", "Dean Huijsen", "13th", "Ruben Amorim", "Anxiety from fans", "Antoine Semenyo", "Keep the ball for a moment", "50%", "Ruben Amorim"],
  "OPTION3": ["3-0", "3 goals", "Antoine Semenyo", "15th", "Assistant Coach", "Injury issues", "Carlos Fernandes", "Defend tightly", "60%", "The players"],
  "OPTION4": ["4-0", "4 goals", "Carlos Fernandes", "20th", "Team Captain", "Weather conditions", "Justin Kluivert", "Play defensively", "70%", "The fans"],
  "OPTION5": ["5-0", "5 goals", "Justin Bieber", "18th", "Fitness Coach", "Poor tactics", "Ruben Amorim", "Attack aggressively", "80%", "The management"],
  "CORRECTOPTION": ["3-0", "2 goals", "Justin Kluivert", "13th", "Ruben Amorim", "Anxiety from fans", "Dean Huijsen", "Keep the ball for a moment", "60%", "Ruben Amorim"],
  "correct_option_type": ["ANY", "ANY", "ANY", "ANY", "ANY", "ANY", "ANY", "ANY", "ANY", "ANY"],
  "MARKS": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  "Chapter Name (Category)": ["", "", "", "", "", "", "", "", "", ""],
  "REASON For the option being the correct answer": ["", "", "", "", "", "", "", "", "", ""],
  "Hint": ["", "", "", "", "", "", "", "", "", ""]
}`;

processJsonToCsv(data);


In [5]:
from pydantic import BaseModel, Field, model_validator
from typing import List, Optional, Dict, Any
    


class UserData(BaseModel):
    user_id: str
    user_name: str
    user_email: str
    api_data: Dict[str, Any] = Field(default_factory=dict)
    user_metadata: Dict[str, Any] = Field(default_factory=dict)

    @model_validator(mode="before")
    def populate_user_metadata(self, values: Dict[str, Any]) -> Dict[str, Any]:
        user_metadata = values.get("user_metadata", {})
        user_metadata["user_id"] = values.get("user_id")
        user_metadata["user_name"] = values.get("user_name")
        user_metadata["user_email"] = values.get("user_email")
        values["user_metadata"] = user_metadata
        return values

class ChatSession(BaseModel):
    session_id: str = Field(..., description="Unique identifier for the WebSocket session.")
    user_data: UserData= Field(default=..., description="User data associated with the session.")
    chat_history: Optional[List[int]] = Field(default=[], description="Conversation history for the session.")


s = ChatSession(session_id="123")
s.user_data = UserData(user_id="123", user_name="Alice", user_email="fdgf", api_data={"a": 1})


ValidationError: 1 validation error for ChatSession
user_data
  Field required [type=missing, input_value={'session_id': '123'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [2]:
d = {
    "user_id": "123",
    "user_name": "Alice",
    "user_email": "dgg"
     }
d.items()

dict_items([('user_id', '123'), ('user_name', 'Alice'), ('user_email', 'dgg')])

In [4]:
for i, k in enumerate(d.items()):
    print(i,k)

0 ('user_id', '123')
1 ('user_name', 'Alice')
2 ('user_email', 'dgg')


In [13]:
from pydantic import BaseModel, Field, model_validator
from typing import List, Optional, Dict, Any
    

class Data(BaseModel):
    data: str
    data2 : str

class UserData(BaseModel):
    user_id: str
    user_name: str
    user_email: str
    api_data: Dict[str, Any] = Field(default_factory=dict)
    user_metadata: Dict[str, Any] = Field(default_factory=dict)
    datas: List[Data] = Field(default_factory=list)


user = UserData(user_id="123", user_name="Alice", user_email="dgg", datas=[Data(data="1", data2="2")])
user.model_dump()




{'user_id': '123',
 'user_name': 'Alice',
 'user_email': 'dgg',
 'api_data': {},
 'user_metadata': {},
 'datas': [{'data': '1', 'data2': '2'}]}